In [81]:
import os
import re
import pandas as pd

In [82]:
import collections

In [83]:
import os
import argparse
import subprocess
from Bio import SeqIO

In [84]:
from itertools import repeat
from multiprocessing import Pool, freeze_support

In [118]:
InDir = "/mnt/d/Lab/16S-Taxa-Phlyo/data/M15"

In [86]:
!ls /mnt/d/Lab/16S-Taxa-Phlyo/data/M15 | wc -l

180


In [123]:
def manifestGen(InDir):
    sampleID = []
    single = {}
    pair = {}
    df = pd.DataFrame()
    SampleList = collections.defaultdict(list)

    for file in os.listdir(InDir):
        filePath = os.path.join(InDir, file)
        if file.endswith(".ab1") and os.path.getsize(filePath) > 0:
            info = file.split("__")
            sampleID.append(info[0])
            SampleList[info[0]].append(filePath)

    for sample in set(sampleID):
        if len(SampleList[sample]) == 1:
            single[sample] = SampleList[sample]
        elif len(SampleList[sample]) == 2:
            pair[sample] = SampleList[sample]
            for end in pair[sample]:
                if "F__" in end:
                    R1 = end
                elif "R__" in end:
                    R2 = end
            df = df.append({"ID": sample, "R1":R1, "R2":R2},ignore_index=True)
    return single, df

In [124]:
single, df = manifestGen(InDir)

In [125]:
single

{'DA_194': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_194__M13R__E10_2004260545Q.ab1'],
 'DA_202': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_202__M13F__B06_2004260550Q.ab1'],
 'DA_186': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_186__M13R__E04_2004260539Q.ab1'],
 'DA_157': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_157__M13R__F07_2004260514Q.ab1'],
 'DA_165': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_165__M13R__C09_2004260527Q.ab1'],
 'DA_149': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_149__M13F__E01_2004260508Q.ab1'],
 'DA_181': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_181__M13R__D03_2004260560Q.ab1'],
 'DA_160': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_160__M13R__A03_2004260557Q.ab1'],
 'DA_115': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_115__M13R__H02_2004260535Q.ab1'],
 'DA_150': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_150__M13R__A07_2004260509Q.ab1'],
 'DA_179': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_179__M13F__D03_2004260520Q.ab1'],
 'DA_184': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_184__M13F__H03

In [121]:
df

,ID,R1,R2
0,DA_141,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_141__M13...,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_141__M13...
1,DA_183,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_183__M13...,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_183__M13...
2,DA_208,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_208__M13...,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_208__M13...
3,DA_177,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_177__M13...,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_177__M13...
4,DA_161,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_161__M13...,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_161__M13...
5,DA_191,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_191__M13...,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_191__M13...
6,DA_139,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_139__M13...,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_139__M13...
7,DA_180,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_180__M13...,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_180__M13...
8,DA_192,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_192__M13...,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_192__M13...
9,DA_199,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_199__M13...,/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_199__M13...


In [145]:
OutDir = "/mnt/d/Lab/16S-Taxa-Phlyo/result/M15"

In [137]:
def parsePairs(df, OutDir):
    R1OutList = []
    R2OutList = []
    df1 = pd.DataFrame()
    fastqDir = os.path.join(OutDir, "fastq")
    if os.path.exists(fastqDir) == 0:
        os.makedirs(fastqDir, 0o777, True)
    for ID in df["ID"]:
        R1OutList.append(os.path.join(fastqDir, ID + "_R1.fastq"))
        R2OutList.append(os.path.join(fastqDir, ID + "_R2.fastq"))
    df1["ID"] = df["ID"]
    df1["R1"] = R1OutList
    df1["R2"] = R2OutList
    return df1

In [138]:
df1 = parsePairs(df)

In [139]:
df1

,ID,R1,R2
0,DA_141,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...
1,DA_183,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...
2,DA_208,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...
3,DA_177,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...
4,DA_161,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...
5,DA_191,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...
6,DA_139,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...
7,DA_180,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...
8,DA_192,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...
9,DA_199,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...


In [24]:
def ABI2Fastq(abi, fastq):
    SeqIO.convert(abi, "abi", fastq, "fastq")

In [25]:
def ABI2FastqParallel(abiList, fastqList):
    pool = Pool(processes = 4)
    pool.starmap(ABI2Fastq, zip(abiList, fastqList))
    pool.close()
    pool.join()
    pool.terminate()

In [120]:
ABI2FastqParallel(df["R1"], R1OutList)

In [121]:
ABI2FastqParallel(df["R2"], R2OutList)

In [122]:
len(R1OutList)

38

In [123]:
len(df)

38

In [62]:
def RunFastp(R1, R2, prefix, OutDir):
    trimDir = os.path.join(OutDir, "trim")
    mergeDir = os.path.join(OutDir, "merge")
    fastpDir = os.path.join(OutDir, "fastp")
    unmergeDir = os.path.join(OutDir, "unmerge")
    if os.path.exists(OutDir) == 0:
        os.makedirs(OutDir, 0o777, True)
    if os.path.exists(trimDir) == 0:
        os.makedirs(trimDir, 0o777, True)
    if os.path.exists(mergeDir) == 0:
        os.makedirs(mergeDir, 0o777, True)
    if os.path.exists(fastpDir) == 0:
        os.makedirs(fastpDir, 0o777, True)
    if os.path.exists(unmergeDir) == 0:
        os.makedirs(unmergeDir, 0o777, True)        
    cmd = "fastp -i " + R1 + " -I " + R2 + " -o " + os.path.join(trimDir, prefix + "_trim_R1.fastq") + " -O " + os.path.join(trimDir, prefix + "_trim_R2.fastq") + \
    " --trim_front1 30 --max_len1 750 --trim_front2 30 --max_len2 750 --cut_front --cut_tail --cut_window_size 20 --cut_mean_quality 30" + \
    " --merge --merged_out " + os.path.join(mergeDir, prefix + ".fastq") + " --correction --overlap_len_require 20 " + \
    " --html " + os.path.join(fastpDir, prefix + ".html") + " --report_title " + prefix + "-fastq-merge-report"
    #print(cmd)
    subprocess.call(cmd, shell=True)

In [63]:
def RunFastpParallel(R1List, R2List, prefixList, OutDir):
    pool = Pool(processes = 4)
    pool.starmap(RunFastp, zip(R1List, R2List, prefixList, repeat(OutDir)))
    pool.close()
    pool.join()
    pool.terminate()

In [136]:
RunFastpParallel(df1["R1"], df1["R2"], df1["ID"], OutDir)

In [5]:
import os

In [3]:
from Bio import SeqIO

In [144]:
InDir = "/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/merge"

In [147]:
def parseFastqPairs(InDir, OutDir):
    allSeq = []
    fastaFileList = []
    fastaDir = os.path.join(OutDir, "fasta")
    if os.path.exists(fastaDir) == 0:
        os.makedirs(fastaDir, 0o777, True)
    for file in os.listdir(InDir):
        if file.endswith(".fastq") and os.path.getsize(os.path.join(InDir, file)) > 0:
            for seq in SeqIO.parse(os.path.join(InDir, file), "fastq"):
                seq.id = file.replace(".fastq", "")
                seq.name = ""
                seq.description = seq.description.split(" ")[1]
                fastaFile = os.path.join(fastaDir, file.replace(".fastq", ".fasta"))
                fastaFileList.append(fastaFile)
                SeqIO.write(seq, fastaFile, "fasta")
                allSeq.append(seq)
    SeqIO.write(allSeq, os.path.join(OutDir, "pairsMerge.fasta"), "fasta")
    return fastaFileList

In [148]:
allSeq = parseFastqPairs(InDir, OutDir)

In [149]:
allSeq

['/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_134.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_136.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_138.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_139.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_141.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_151.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_152.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_154.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_156.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_158.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_161.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_163.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_164.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_166.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_168.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fasta/DA_169.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/f

In [155]:
indexPath = "/mnt/d/Lab/16S-Taxa-Phlyo/database/silva-138-99-index.tsv"

In [159]:
def parseIndex(blastDir, OutDir):
    taxaList = []
    df = pd.DataFrame()
    out = pd.DataFrame()
    for file in os.listdir(blastDir):
        #print(file)
        if file.endswith(".tsv") and os.path.getsize(os.path.join(blastDir, file)) > 0:
            df1 = pd.read_table(os.path.join(blastDir, file), header = None)
            df1.columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
            df = df.append(df1)
    df = df.reset_index()
    df2 = pd.read_table(indexPath)
    index = dict(zip(df2["ID"], df2["Taxa"]))
    for i in range(len(df)):
        taxa = index[df["sseqid"][i]]
        taxaList.append(taxa)
    out["taxa"] = taxaList
    out["qseqid"] = df["qseqid"]
    out["pident"] = df["pident"]
    out["length"] = df["length"]
    return out

In [167]:
df1[0:1]

,ID,R1,R2
0,DA_141,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...,/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/fastq/DA_...


In [160]:
OutDir

'/mnt/d/Lab/16S-Taxa-Phlyo/result/M15'

In [161]:
pairsOut = parseIndex(os.path.join(OutDir, "blast"), OutDir)

In [162]:
pairsOut

,taxa,qseqid,pident,length
0,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,DA_134,99.637,1379
1,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,DA_136,98.908,1374
2,Bacteria;Firmicutes;Bacilli;Lactobacillales;La...,DA_138,100.000,1422
3,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,DA_139,99.708,1371
4,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,DA_141,99.855,1379
5,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,DA_151,98.983,1376
6,Bacteria;Actinobacteriota;Coriobacteriia;Corio...,DA_152,99.926,1354
7,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,DA_154,99.636,1372
8,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,DA_156,99.637,1377
9,Bacteria;Bacteroidota;Bacteroidia;Bacteroidale...,DA_158,99.420,1380
